In [1]:
!pip install pandarallel
import pandas as pd
pd.set_option("max_colwidth", 600)
from bs4 import BeautifulSoup
import requests

from pandarallel import pandarallel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.2-py3-none-any.whl size=16411 sha256=372debcbacfd313351a42d09955c46c74dde47596b276f31e98f2bdd2e6cdd2e
  Stored in directory: /root/.cache/pip/wheels/01/8d/99/4ce6a1e914ed6ec383bc099dfbe0112dfde0bc39957ca93ec1
Successfully built pandarallel


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
def extract_urls_topic(url):
    try:
      coverpage = requests.get(url).content
      soup = BeautifulSoup(coverpage, 'html.parser')
     
      rows = []
      coverpage=soup.find_all('div',class_='archive-item')
      for link in coverpage:
        rows.append('https://www.elprogreso.es'+link.a['href']) 
    except:
      rows = 'error'
      
    return rows

In [8]:
extract_urls_topic('https://www.elprogreso.es/blog/section/mundo?page=795')

['https://www.elprogreso.es/articulo/mundo/israel-declara-o-estado-de-excepcion-na-fronteira-con-gaza/20070520150800147212.html',
 'https://www.elprogreso.es/articulo/mundo/detido-kasparov-no-aeroporto-de-moscu-antes-de-participar-nunha-protesta-contra-o/20070518131700147374.html',
 'https://www.elprogreso.es/articulo/mundo/francois-fillon-asume-o-cargo-de-primeiro-ministro-coa-intencion-de-servir-francia/20070517151800147301.html',
 'https://www.elprogreso.es/articulo/mundo/duas-coreas-unense-por-un-tren-nun-novo-paso-cara-unificacion/20070517134300147973.html',
 'https://www.elprogreso.es/articulo/mundo/hamas-e-al-fatah-anuncian-un-alto-o-fogo-en-palestina/20070516145700149817.html',
 'https://www.elprogreso.es/articulo/mundo/kostunica-e-reelexido-como-primeiro-ministro-serbio-polo-parlamento/20070516144100148542.html',
 'https://www.elprogreso.es/articulo/mundo/sarko-president-man-de-ferro-colle-chaves-do-eliseo/20070516125100149222.html',
 'https://www.elprogreso.es/articulo/mundo/

In [9]:
pandarallel.initialize(nb_workers= 30, progress_bar=True)
df = pd.DataFrame(range(1,2348),columns = ['page'])
url_str=''

for i in range(1,796) :
    url_str='https://www.elprogreso.es/blog/section/mundo?page='+str(i)
    df.page[i] = url_str

for i in range(1,1552) :
    url_str='https://www.elprogreso.es/blog/section/espana?page='+str(i)
    df.page[i+795] = url_str

df['rows']=df['page'].parallel_apply(extract_urls_topic)
df = df[df.rows.str.len()!=0].reset_index(drop=True)
df = df.explode('rows', ignore_index=True)
df=df.drop_duplicates(subset='rows')
df = df.dropna()
print(df)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/elprogreso/url.csv',index=False)

INFO: Pandarallel will run on 30 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


                                                          page  \
0                                                            1   
1          https://www.elprogreso.es/blog/section/mundo?page=1   
2          https://www.elprogreso.es/blog/section/mundo?page=1   
3          https://www.elprogreso.es/blog/section/mundo?page=1   
4          https://www.elprogreso.es/blog/section/mundo?page=1   
...                                                        ...   
46899  https://www.elprogreso.es/blog/section/espana?page=1551   
46900  https://www.elprogreso.es/blog/section/espana?page=1551   
46901  https://www.elprogreso.es/blog/section/espana?page=1551   
46902  https://www.elprogreso.es/blog/section/espana?page=1551   
46903  https://www.elprogreso.es/blog/section/espana?page=1551   

                                                                                                                                                            rows  
0                                           